In [2]:
# Import scraping modules
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Import data manipulation modules
import pandas as pd
import numpy as np
# Import data visualization modules
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
import requests

In [3]:
df_game_data = pd.read_csv('../documents/nfl_pass_rush_receive_raw_data.csv')
df_game_data

/tmp/ipykernel_9751/3054359270.py:1: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df_game_data = pd.read_csv('../documents/nfl_pass_rush_receive_raw_data.csv')


,game_id,player_id,pos,player,team,pass_cmp,pass_att,pass_yds,pass_td,pass_int,...,OT,Roof,Surface,Temperature,Humidity,Wind_Speed,Vegas_Line,Vegas_Favorite,Over_Under,game_date
0,201909050chi,RodgAa00,QB,Aaron Rodgers,GNB,18,30,203,1,0,...,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
1,201909050chi,JoneAa00,RB,Aaron Jones,GNB,0,0,0,0,0,...,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
2,201909050chi,ValdMa00,WR,Marquez Valdes-Scantling,GNB,0,0,0,0,0,...,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
3,201909050chi,AdamDa01,WR,Davante Adams,GNB,0,0,0,0,0,...,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
4,201909050chi,GrahJi00,TE,Jimmy Graham,GNB,0,0,0,0,0,...,False,outdoors,grass,65,69,10,-3.5,CHI,47.0,2019-09-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26595,202301080was,ThomLo00,TE,Logan Thomas,WAS,0,0,0,0,0,...,False,outdoors,grass,42,53,7,NaN,NaN,NaN,2023-01-08
26596,202301080was,SimsCa00,WR,Cam Sims,WAS,0,0,0,0,0,...,False,outdoors,grass,42,53,7,NaN,NaN,NaN,2023-01-08
26597,202301080was,SamuCu00,WR,Curtis Samuel,WAS,0,0,0,0,0,...,False,outdoors,grass,42,53,7,NaN,NaN,NaN,2023-01-08
26598,202301080was,BonnRe00,RB,Reggie Bonnafon,WAS,0,0,0,0,0,...,False,outdoors,grass,42,53,7,NaN,NaN,NaN,2023-01-08


In [4]:
len(df_game_data['player_id'].unique())

1156

In [5]:
all_players = df_game_data['player_id'].unique()
all_players.sort()
df_game_data[df_game_data['player_id']=='BradTo00']


,game_id,player_id,pos,player,team,pass_cmp,pass_att,pass_yds,pass_td,pass_int,...,OT,Roof,Surface,Temperature,Humidity,Wind_Speed,Vegas_Line,Vegas_Favorite,Over_Under,game_date
210,201909080nwe,BradTo00,QB,Tom Brady,NWE,24,36,341,3,0,...,False,outdoors,grass,66,62,2,-5.5,NWE,49.0,2019-09-08
555,201909150mia,BradTo00,QB,Tom Brady,NWE,20,28,264,2,0,...,False,outdoors,grass,87,67,4,-18.0,NWE,48.5,2019-09-15
977,201909220nwe,BradTo00,QB,Tom Brady,NWE,28,42,306,2,0,...,False,outdoors,grass,81,49,10,-20.5,NWE,43.0,2019-09-22
1202,201909290buf,BradTo00,QB,Tom Brady,NWE,18,39,150,0,1,...,False,outdoors,astroturf,58,71,10,-7.0,NWE,41.5,2019-09-29
1787,201910060was,BradTo00,QB,Tom Brady,NWE,28,42,348,3,1,...,False,outdoors,grass,70,66,8,-15.5,NWE,42.0,2019-10-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24997,202212110sfo,BradTo00,QB,Tom Brady,TAM,34,55,253,1,2,...,False,outdoors,grass,54,54,8,NaN,NaN,NaN,2022-12-11
25365,202212180tam,BradTo00,QB,Tom Brady,TAM,30,44,312,3,2,...,False,outdoors,grass,66,43,6,NaN,NaN,NaN,2022-12-18
25721,202212250crd,BradTo00,QB,Tom Brady,TAM,32,48,281,1,2,...,True,retractable roof (open),grass,72,45,0,NaN,NaN,NaN,2022-12-25
26154,202301010tam,BradTo00,QB,Tom Brady,TAM,34,45,432,3,0,...,False,outdoors,grass,75,87,2,NaN,NaN,NaN,2023-01-01
